In [ ]:
# %pip install python-dotenv

In [4]:
import os
import json
from dotenv import load_dotenv

import pandas as pd
from sqlalchemy import create_engine

#### Check if data files have been extracted

In theory, this notebook should be run after `EDA.ipynb` where we perform our intial data exploratory.
In that notebook, we unzip the compressed data file into two `.ndjson` for submission and comment data.

In [5]:
comments_data_path = os.path.join("..", "data", "extracted_data", "comments_data.ndjson")
submissions_data_path = os.path.join("..", "data", "extracted_data", "submissions_data.ndjson")

assert os.path.isfile(comments_data_path) is True
assert os.path.isfile(submissions_data_path) is True

In [10]:
def upload_to_db(db_engine, data_file: str, table_name: str, selected_columns: list[str]):
    """
    Upload 
    """

    assert os.path.isfile(data_file) is True
    assert data_file.endswith('.ndjson') is True

    with open(data_file, 'r') as f:
        done = False
        it = 1
        while not done:
            jsons = []
            # 1000 lines at a time
            for i in range(1000):
                line = f.readline().strip()

                # end of line is reached
                if not line:
                    done = True
                    break

                json_line = json.loads(line)
                jsons.append(json_line)
            
            if done:
                break

            df = pd.DataFrame(jsons)
            # only select relevant columns
            for col in selected_columns:
                # NOTE: some json entries may not contain the selected column,  
                # we will that as NA
                if col not in df.columns:
                    df[col] = pd.NA

            df = df[selected_columns]
            df.to_sql(table_name, db_engine, if_exists='append', index=False)
            print(f'iteration #{it}: data written to {table_name} successfully.')
            iter += 1

#### Create database engine

In [8]:
dotenv_path = '../.env'
load_dotenv(dotenv_path)

def get_db_url(db_name):
    db_str = 'postgresql+psycopg2://{}:{}@{}:{}/{}'
    return db_str.format(
        os.getenv('DB_USERNAME'),
        os.getenv('DB_PASSWORD'),
        os.getenv('DB_HOST'),
        os.getenv('DB_PORT'),
        db_name
    )

db_url = get_db_url('reddit')
db_engine = create_engine(db_url)

# TODO: ensure db_engine is successfully created

#### Upload data to database

Comments:

In [ ]:
# define relevant comment columns
# this is the result of the EDA we did on the Reddit dataset
relevant_comment_columns = [
    'author',
    'author_created_utc',
    'body',
    'created_utc',
    'id',
    'locked',
    'parent_id',
    'permalink',
    'retrieved_on',
    'score',
    'subreddit',
    'subreddit_id',
    'subreddit_name_prefixed',
    'subreddit_type',
    'archived',
    'downs',
    'updated_on',
    'ups'
]

upload_to_db(db_engine, comments_data_path, 'comments', relevant_comment_columns)

Submissions:

In [ ]:
# define relevant submission columns
# this is the result of the EDA we did on the Reddit dataset
relevant_submission_columns = [
    "id", "downs", "ups", "archived", "author", 
    "author_created_utc", "subreddit", "subreddit_id", 
    "subreddit_subscribers", "subreddit_type", "title", 
    "url", "num_comments", "permalink", 
    "is_self", "selftext", "created_utc", "spoiler", "locked"
]

upload_to_db(db_engine, submissions_data_path, 'submissions', relevant_submission_columns)